In [26]:
import scqubits as scqubits
import autograd.numpy as anp
from autograd import grad, jacobian
import scipy.optimize as optimize
import scipy.linalg as linalg
import qutip as qt
import matplotlib.pyplot as plt
from cosmatrix import cosm
from autograd.extend import primitive, defvjp
import os
import dynamiqs as dq
from swt import swt_subspace

# Disable multi-threading to avoid issues with fork
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable GPU if present




class Hamiltonian:
    def __init__(self, phi_ex, Ej, Ec, bare_dim, trunc_dim, omega_c1, omega_c2, phizpf_dependence = True, twolevelapp = False):
        self.phi_ex = phi_ex
        self.Ej = Ej * 2*anp.pi
        self.Ec = Ec * 2*anp.pi
        self.beta = 1
        self.N = 1
        self.original_dim = bare_dim
        self.trunc_dim = trunc_dim
        self.sdim, self.c2dim, self.c1dim = bare_dim
        self.total_dim = self.sdim*self.c2dim*self.c1dim
        self.omega_c1 = omega_c1
        self.omega_c2 = omega_c2
        self.phizpf_dependence = phizpf_dependence
        self.twolevelapp = twolevelapp
        self.g = 0.05*2*anp.pi
        self.option = dq.Options(progress_meter=None)
        self.H , self.H_control = self.get_H()
        self.H_dressed, self.H_control_dressed = self.dressed_basis()
        

    def sort_eigenpairs(self, eigenvalues, eigenvectors):
        n = eigenvectors.shape[0]
        sorted_indices = []

        for i in range(n):
            max_abs_vals = anp.abs(eigenvectors[i, :])
            remaining_indices = [j for j in range(len(max_abs_vals)) if j not in sorted_indices]
            max_index = remaining_indices[anp.argmax(max_abs_vals[remaining_indices])]
            sorted_indices.append(max_index)

        sorted_indices_array = anp.array(sorted_indices)
        sorted_eigenvalues = eigenvalues[sorted_indices_array]
        sorted_eigenvectors = eigenvectors[:, sorted_indices_array]
        return sorted_eigenvalues, sorted_eigenvectors
    
    def state_index(self, index,dim):
        n,k,m = index
        N,K,M = dim
        return n*K*M + k*M + m
    
    def get_indices(self, trunc):
        """
        Convert a list of 3D indices to 1D indices based on given dimensions.
        indecies follow (squid, cavity2, cavity1)
        
        Args:
            trunc: List of three integers [N,K,M] specifying the dimensions to include
                 N is number of SNAIL states
                 K is number of cavity 2 states
                 M is number of cavity 1 states
                 
        Returns:
            List of 1D indices corresponding to the 3D indices
        """
        N, K, M = trunc
        indices_1d = []
        
        # Generate all 3D indices and store indices outside truncation
        for n in range(self.sdim):
            for k in range(self.c2dim):
                for m in range(self.c1dim):
                    # Only add indices that are outside the truncation
                    if n >= N or k >= K or m >= M:
                        index = [n,k,m]
                    
                        # Convert to 1D index using state_index method
                        idx = self.state_index(index, self.original_dim)
                        indices_1d.append(idx)
        return indices_1d
        
    def remove_row_col(self, matrix, indices):
        # Check if matrix is square
        if matrix.shape[0] != matrix.shape[1]:
            raise ValueError("Input matrix must be square")
        
        # Convert single index to list if needed
        if isinstance(indices, (int, anp.integer)):
            indices = [indices]
            
        # Convert to list if numpy array
        if isinstance(indices, anp.ndarray):
            indices = indices.tolist()
            
        # Sort indices in descending order to avoid changing indices when deleting
        indices = sorted(indices, reverse=True)
        
        # Check if indices are valid
        if any(i < 0 or i >= matrix.shape[0] for i in indices):
            raise ValueError(f"All indices must be between 0 and {matrix.shape[0]-1}")
        
        # Remove rows and columns
        # Don't use copy as it breaks autograd's gradient tracking
        result = matrix  # Start with the original matrix
        # anp.delete is not compatible with autograd, so we need to implement
        # row/column deletion manually using slicing and concatenation
        for idx in indices:
            # Remove row
            if idx > 0:
                result = anp.concatenate((result[:idx], result[idx+1:]), axis=0)
            else:
                result = result[1:]
                
            # Remove column
            if idx > 0:
                result = anp.concatenate((result[:, :idx], result[:, idx+1:]), axis=1)
            else:
                result = result[:, 1:]
        return result
    
    def truncate_matrix(self, matrix, trunc):
        indices = self.get_indices(trunc)
        return self.remove_row_col(matrix, indices)
    
    def annihilation(self, dim):
        return anp.diag(anp.sqrt(anp.arange(1,dim)),1)
    def creation(self, dim):
        return anp.diag(anp.sqrt(anp.arange(1,dim)),-1)
    def tensor_product(self,A,B,C):
        return anp.kron(A,anp.kron(B,C))
    
    def SNAIL(self):
        """
        This function starts from SNAIL hamiltonian in oscillator basis,  and then calculates the SNAIL Hamiltonian and returns the Hamiltonian and the charge operator in eigenbasis. 
        returns: Hs, charge_op in eigenbasis
        """
        beta = self.beta
        N = self.N
        Ej = self.Ej
        Ec = self.Ec
        phi_ex = self.phi_ex*2*anp.pi
        sdim = 20
        # Find minimum of SNAIL potential using autograd primitive
        @primitive
        def Us_min(phi_ex):
            def U_s(phi): 
                return (-beta*anp.cos(phi-phi_ex)-N*anp.cos((phi)/N))
            phi_min = optimize.minimize(U_s, anp.array([0.])).x
            return phi_min
        
        def phi_minde(ans, phi_ex):
            def phi_minde_vjp(g):
                c2 = beta*anp.cos(ans - phi_ex) + 1/N*anp.cos(ans/N)
                return g*beta*anp.cos(ans - phi_ex)/c2
            return phi_minde_vjp
        defvjp(Us_min, phi_minde)
        
        phi_min = Us_min(phi_ex)
        # potential expansion around minimum
        c2 = beta*anp.cos(phi_min - phi_ex) + 1/N*anp.cos(phi_min/N)
        omega_s = anp.sqrt(8*c2*Ej*Ec)
        if self.phizpf_dependence == True:
            phi_zpf = anp.power(2*Ec/(Ej*c2),1/4)
        else:
            phi_zpf = anp.power(2*Ec/(Ej*c2),1/4)
        self.phi_zpf = phi_zpf
        g2 = Ej*phi_zpf**2*c2/2
        s = self.annihilation(sdim)
        sd = self.creation(sdim)
        x2 = anp.matmul(s+sd,s+sd)
        Hs =( omega_s * anp.matmul(sd,s)
            - Ej*(beta*cosm(phi_zpf*(s+sd)+(phi_min-phi_ex)*anp.identity(sdim))
            + N*cosm((phi_zpf*(s+sd)+phi_min*anp.identity(sdim))/N))- g2*x2)
        Hs = anp.real(Hs)
        charge_op = -1j*(s-sd)/2/phi_zpf
        energy0,U = anp.linalg.eigh(Hs)
        energy0,U = self.sort_eigenpairs(energy0, U)
        energy0 = energy0 - energy0[0]
        Ud = anp.conjugate(anp.transpose(U))
        Hs = Ud@Hs@U
        
        charge_op = Ud@charge_op@U
        Hs = Hs - Hs[0,0]*anp.identity(sdim)
        Hs = Hs[0:self.sdim,0:self.sdim]
        charge_op = charge_op[0:self.sdim,0:self.sdim]
        return Hs,charge_op
    
    def get_H(self):
        """This function calculates the composite system Hamiltonian and returns the Hamiltonian and the SNAIL charge operator in composite Hilbert space. Returned matrix is under eigenbasis of each subsystem
        returns: H = Hc1 + Hc2 + Hs + H_int1 + H_int2, charge_op in composite Hilbert space
        """
        c1dim = 20
        c2dim = 20

        Hs,charge_op, = self.SNAIL()
        squid = [Hs,charge_op]
        cavity = scqubits.Oscillator(
            E_osc=self.omega_c1,
            truncated_dim = c1dim
        )
        Hc = anp.diag(anp.array(cavity.eigenvals(evals_count=c1dim) - cavity.eigenvals(evals_count=c1dim)[0]))*2*anp.pi
        Hc = Hc[0:self.c1dim,0:self.c1dim]
        Vc = anp.array(cavity.creation_operator() + cavity.annihilation_operator())
        Vc = Vc[0:self.c1dim,0:self.c1dim]
        cavity = [Hc,Vc]

        cavity2 = scqubits.Oscillator(
            E_osc=self.omega_c2,
            truncated_dim = c2dim
        )
        Hc2 = anp.diag(anp.array(cavity2.eigenvals(evals_count=c2dim) - cavity2.eigenvals(evals_count=c2dim)[0]))*2*anp.pi
        Hc2 = Hc2[0:self.c2dim,0:self.c2dim]
        Vc2 = anp.array(cavity2.creation_operator() + cavity2.annihilation_operator())
        Vc2 = Vc2[0:self.c2dim,0:self.c2dim]
        cavity2 = [Hc2,Vc2]
        H,H_control = self.composite_sys(squid,cavity,cavity2)
        return H,H_control

    def composite_sys(self, squid,cavity, cavity2,):
        Hs, charge_op = squid
        Hc, Vc = cavity
        Hc2, Vc2 = cavity2
        sdim = Hs.shape[0]
        cdim = Hc.shape[0]
        cdim2 = Hc2.shape[0]
        Ic = anp.identity(cdim)
        Ic2 = anp.identity(cdim2)
        Is = anp.identity(sdim)
        Hs = self.tensor_product(Hs,Ic,Ic2)
        Hc = self.tensor_product(Is, Hc, Ic2)
        Hc2 = self.tensor_product(Is, Ic, Hc2)
        H_int = self.g * self.tensor_product(charge_op, Vc, Ic2) 
        H_int2 = self.g * self.tensor_product(charge_op, Ic, Vc2) 
        H = Hs + Hc + Hc2 + H_int + H_int2
        H_control = self.tensor_product(charge_op, Ic, Ic2)
        return H, H_control

    def dressed_basis(self):
        H, H_control = self.get_H()
        evals, U = anp.linalg.eigh(H)
        evals, U = self.sort_eigenpairs(evals, U)
        evals = evals - evals[0]
        Ud = anp.conjugate(anp.transpose(U))

        H_dressed = Ud @ H @ U  # Keep this as diagonalized H
        H_dressed = H_dressed - H_dressed[0, 0] * anp.identity(self.total_dim)  # Shift to zero
        H_control_dressed = Ud @ H_control @ U
        return H_dressed, H_control_dressed

    
    def omegasder(self, ):
        Hs, _ = self.SNAIL()
        """
        Calculate the energy of the first excited state (omegas) and its derivative with respect to phi_ex
        
        Returns:
            Tuple containing (omegas, derivative of omegas)
        """
        # Get the energy of the first excited state
        omegas = Hs[1, 1]
        
        # Define a function that computes the first excited state energy for a given phi
        def energy_level(phi):
            sc = Hamiltonian(phi, self.Ej/(2*anp.pi), self.Ec/(2*anp.pi), 
                           self.original_dim, self.trunc_dim, self.omega_c1, 
                           self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            H, _ = sc.SNAIL()
            return anp.real(H[1, 1])  # Return the energy of the first excited state
        
        # Compute derivative of omegas with respect to phi_ex
        derivative = grad(energy_level)(self.phi_ex)
        
        return omegas, derivative

    def der_formula(self, A, omegad):
        Hs, _ = self.SNAIL()
        _, omegasder = self.omegasder()
        omegas = Hs[1,1]
        anh = -self.Ec
        g = self.g/2/self.phi_zpf
        A = A/2/self.phi_zpf
        Delta = omegas - self.omega_c1*2*anp.pi
        omegas += g**2/Delta
        delta = omegas - omegad
        chi = g**2/Delta**2*anh*2
        delta1 = delta + chi
        # der = g**2/Delta**2*(1 - 2*A**2*anh*(delta + delta1)/(delta*delta1)**2/4)
        der = g**2/Delta**2*(1 - A**2/delta**3*anh)
        return der*omegasder
    
    def keep_same_elements(self, target, given):
        """
        Modifies the given matrix to have non-zero elements only where the target matrix has non-zero elements.
        
        Args:
            target: The reference matrix that defines the non-zero pattern
            given: The matrix to be modified
            
        Returns:
            A modified version of the given matrix with zeros where target has zeros
        """
        # Create a mask where target is non-zero
        mask = anp.abs(target) > 1e-10
        # Apply the mask to the given matrix
        return anp.where(mask, given, 0.0)
    

    def HR(self, A, omegad):
        H0, Hc = self.H_dressed, self.H_control_dressed
        s = self.annihilation(self.sdim)
        sd = self.creation(self.sdim)
        Ic1 = anp.identity(self.c1dim)
        Ic2 = anp.identity(self.c2dim)
        H0 = H0 - self.tensor_product(omegad*sd@s, Ic1, Ic2)
        target = self.tensor_product(sd+s, Ic1, Ic2)
        Hc = self.keep_same_elements(target, Hc)
        if self.twolevelapp == True:
            H0 = self.truncate_matrix(H0, [2, self.trunc_dim[1], self.trunc_dim[2]])
            Hc = self.truncate_matrix(Hc, [2, self.trunc_dim[1], self.trunc_dim[2]])
        else:
            H0 = self.truncate_matrix(H0, self.trunc_dim)
            Hc = self.truncate_matrix(Hc, self.trunc_dim)
        self.H_dressed_truncated = H0
        self.Hcapp = Hc
        H = H0 + A/2*Hc
        eigvals, _ = anp.linalg.eigh(H)
        eigvals, _ = self.sort_eigenpairs(eigvals, _)
        eigvals = eigvals - eigvals[0]
        
        # Return the first few energy levels based on truncation dimensions
        if self.trunc_dim[1] == 1:
            return eigvals[1], eigvals[2], eigvals[3]
        else:
            return eigvals[1], eigvals[self.trunc_dim[2]]
    
    def approximateder(self, A=0.0, omegad=0.0):
        """
        Calculate the derivative of the rotating frame Hamiltonian energies with respect to phi_ex
        
        Args:
            A: Drive amplitude
            omegad: Drive frequency
            
        Returns:
            Array of derivatives for each energy level
        """
        # Define functions that compute individual energy levels for a given phi
        def energy_level_1(phi):
            sc = Hamiltonian(phi, self.Ej/(2*anp.pi), self.Ec/(2*anp.pi), 
                            self.original_dim, self.trunc_dim, self.omega_c1, 
                            self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            levels = sc.HR(A, omegad)

            return levels[0]  # First energy level
            
        def energy_level_2(phi):
            sc = Hamiltonian(phi, self.Ej/(2*anp.pi), self.Ec/(2*anp.pi), 
                            self.original_dim, self.trunc_dim, self.omega_c1, 
                            self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            levels = sc.HR(A, omegad)
            return levels[1]  # Second energy level
            
        def energy_level_3(phi):
            sc = Hamiltonian(phi, self.Ej/(2*anp.pi), self.Ec/(2*anp.pi), 
                            self.original_dim, self.trunc_dim, self.omega_c1, 
                            self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            levels = sc.HR(A, omegad)
            if len(levels) > 2:
                return levels[2]  # Third energy level if available
            return 0.0
        
        # Compute derivatives for each energy level individually
        der1 = jacobian(energy_level_1)(self.phi_ex)
        der2 = jacobian(energy_level_2)(self.phi_ex)
        der3 = jacobian(energy_level_3)(self.phi_ex)
        
        # Return the derivatives as a tuple
        if anp.all(der3 == 0.0):
            return der1, der2
        else:
            return der1, der2, der3
    
        
    def static_rate(self, der):
        A = 5e-6
        # Handle the case where der is a tuple of derivatives
        if isinstance(der, tuple):
            # Process each derivative in the tuple
            results = []
            for d in der:
                results.append(anp.abs(d*4.4)*1e6*A)
            return results
        else:
            # Handle single derivative case
            return anp.abs(der*4.4)*1e6*A
    
    def obtain_omegad(self, A, ):
        Hs, _ = self.SNAIL()
        omegas = Hs[1,1]
        anh = -self.Ec
        g = self.g/2/self.phi_zpf
        A = A/2/self.phi_zpf/2
        Delta = omegas - self.omega_c1*2*anp.pi
        omegas += g**2/Delta
        delta=-(anp.abs(anh*A**2))**(1/3)
        return omegas-delta
    
    def find_optimal_omega_d(self, A, initial_omega_d=6.3*2*anp.pi, bounds=(6.0*2*anp.pi, 6.4*2*anp.pi)):
        from scipy.optimize import minimize_scalar
        
        # Function to minimize
        def objective(omega_d):
            return anp.abs(self.approximateder(A, omega_d)[0])
        
        # Find the minimum
        result = minimize_scalar(objective, bracket=bounds, method='brent')
        
        # Return the optimal omega_d and the corresponding minimum value
        return result.x, result.fun

In [27]:
#check approximations
from joblib import Parallel, delayed

Ej = 30.19
Ec = 0.1
omega_c1 = 5.226
omega_c2 = 8.135
phi_ex = 0.2
bare_dim = [10,1,6]
trunc_dim = [5,1,4]
# Define the combinations of parameters to test



# First subplot - A = 0.5e-4*2*anp.pi
A = 10e-3*2*anp.pi
omega_ds1 = anp.linspace(6.15, 6.17, 200)*2*anp.pi
phizpf_dependence = True
twolevelapp = False

sc = Hamiltonian(phi_ex, Ej, Ec, bare_dim, trunc_dim, omega_c1, omega_c2, phizpf_dependence, twolevelapp)

In [29]:
optimal_omegad, value = sc.find_optimal_omega_d(A)